In [1]:
import urllib.request
from zipfile import ZipFile
import pandas as pd
import calendar
import os

In [2]:
def download_from_url(url, file_name):
    urllib.request.urlretrieve(url, file_name)

In [3]:
def extract_from_zip_folder(file_type, zip_folder_path, new_folder_path):
    '''This function extracts files from a zip folder.
    It just extracts the files of a certain type
    '''
    with ZipFile(zip_folder_path, 'r') as zip_obj:
        # Get a list of all archived file names from the zip
        list_of_file_names = zip_obj.namelist()
        # Iterate over the file names
        for file_name in list_of_file_names:
            # Check filename endswith csv
            if file_name.endswith(f'.{file_type}'):
                # Extract a single file from zip
                print(f'extracting {file_name}')
                zip_obj.extract(file_name, new_folder_path)

In [4]:
def add_missing_days(dataframe, partition_column):
    '''This function takes the monthly dataframe and checks if there are missing days
    for a specific sample spot. If so, it appends a row to the original dataframe with
    the info of that day and the validation columns set to N (non validated), to know that
    info is not correct (we will correct it later).
    '''
    year = dataframe.loc[0, 'ANO']
    month = dataframe.loc[0, 'MES']
    
    # First we have to know how many days a specific month has:
    _, number_days_month = calendar.monthrange(year, month)
    
    # We create a list with all the days of that month
    list_of_days_of_the_month = list(range(1, number_days_month + 1))
    
    # We create a list with all the sample spots
    sample_spots_list = list(set(dataframe[partition_column]))
    
    for sample_spot in sample_spots_list:
        print(f'Checking sample spot: {sample_spot}')
        # We create a df with just the info of one spot
        sample_spot_df = dataframe[dataframe[partition_column] == sample_spot].reset_index(drop=True)

        # We check if all that days are contained in the spot df
        isin_df = pd.Series(list_of_days_of_the_month).isin(list(sample_spot_df['DIA']))
        isin_df.index = list_of_days_of_the_month

        # Now, if a day is not included, we append a row with its data to the original df
        for day, isin in isin_df.iteritems():
            if isin == False:
                print(f'Day {day}-{month}-{year} missing')
                
                # We take the first row of the df, but we change the day and the validation columns to 'N'
                row_to_append = [sample_spot_df.loc[0, column] for column in sample_spot_df.columns]
                row_to_append[6] = day
                for i, e in enumerate(row_to_append):
                    if e == 'V':
                        row_to_append[i] = 'N'
                print(row_to_append)
                
                # We append the row
                dataframe = dataframe.append(pd.Series(row_to_append, index=sample_spot_df.columns), ignore_index=True)
                print(f'Day {day}-{month}-{year} row added to original dataframe')
            
    return dataframe

In [5]:
def get_stacked_dataframe(dataframe, cols_to_drop, cols_remain):
    '''This function applies the pandas stack method to make data that is
    spread in columns collapse in a single column.
    First drops the columns that would not let the stack work properly,
    as we want to have the columns that will remain as they are,
    and the columns that will be stacked in the same column.
    Then sets the columns that do not have to be stacked as the index.
    Applies stack method. Finally, resets index.
    '''
    dataframe = dataframe.drop(columns=cols_to_drop).set_index(cols_remain)
    dataframe = dataframe.stack().reset_index()
    dataframe = dataframe[dataframe[dataframe.columns[-2]] != 'index'].reset_index(drop=True)
    return dataframe

In [6]:
def add_last_col_to_df(df1, df2):
    ''' Adds the last column from a dataframe to another dataframe with the same number of rows'''
    df1['new_col'] = df2.iloc[:,-1]
    return df1

In [7]:
def get_next_validated_value(dataframe, index, column, get_next=True):
    '''This function gets the next or previous validated row index in a dataframe'''
    iterator = 0
    next_validated = 'N'
    
    while next_validated != 'V':
        iterator += 1
        if get_next:
            next_validated = dataframe.loc[index + iterator, column]
            wanted_index = index + iterator
        else:
            next_validated = dataframe.loc[index - iterator, column]
            wanted_index = index - iterator

    return wanted_index

In [8]:
def assign_non_validated_values(dataframe, partition_column):
    '''This function partitions the dataframe by a partition column (PUNTO_MUESTREO)
    into smaller dataframes and, for each of them, sets new values to non validated records,
    based on near values. It tries to get the nearest next and previous validated values,
    to assign an average of them. If it doesn't find a nearest next validated value,
    it assigns the nearest previous one, and viceversa.
    '''
    dataframe = dataframe.sort_values(by=['PUNTO_MUESTREO', 'ANO', 'MES', 'DIA', 'HORA']).reset_index(drop=True)
    
    sample_spots_list = list(set(dataframe[partition_column]))
    
    for sample_spot in sample_spots_list:
        sample_spot_df = dataframe[dataframe[partition_column] == sample_spot]
        sample_spot_df_n = sample_spot_df[sample_spot_df['VALIDADO'] == 'N']
        
        for index, row in sample_spot_df_n.iterrows():
            print(f"Reassigning NO2 value for spot {row[partition_column]}, {row['ANO']}-{row['MES']}-{row['DIA']} {row['HORA']}:00")
            try:
                next_validated_index = get_next_validated_value(sample_spot_df, index, 'VALIDADO')
                
                try:
                    previous_validated_index = get_next_validated_value(sample_spot_df, index, 'VALIDADO', get_next=False)
                    dataframe.loc[index, 'NIVEL_NO2'] = (
                        sample_spot_df.loc[next_validated_index, 'NIVEL_NO2'] +
                        sample_spot_df.loc[previous_validated_index, 'NIVEL_NO2']
                    ) / 2
                
                except KeyError:
                    next_validated_index = get_next_validated_value(sample_spot_df, index, 'VALIDADO')
                    dataframe.loc[index, 'NIVEL_NO2'] = sample_spot_df.loc[next_validated_index, 'NIVEL_NO2']
            
            except KeyError:
                next_validated_index = get_next_validated_value(sample_spot_df, index, 'VALIDADO', get_next=False)
                dataframe.loc[index, 'NIVEL_NO2'] = sample_spot_df.loc[next_validated_index, 'NIVEL_NO2']
                
            dataframe.loc[index, 'VALIDADO'] = 'R'

    return dataframe

In [9]:
def get_clean_df(dataframe):
    '''Gets a df, keeps just the NO2 info, splits it into 2 dataframes,
    each of them with one of the columns that we want to stack,
    joins them into a single dataframe, renames columns and formats HORA column.
    The result is a much easier to use dataframe'''
    
    print(f'Dataframe shape: {dataframe.shape}')
    
    print('Keeping just NO2 data')
    dataframe = dataframe[dataframe['MAGNITUD'] == 8].drop(columns=['MAGNITUD']).reset_index(drop=True)
    
    print(f'Dataframe shape: {dataframe.shape}')
    
    print('Adding missing days rows')
    # We need a list of the sample spots
    list_of_sample_spots = list(set(dataframe['PUNTO_MUESTREO']))
    print(f'{len(list_of_sample_spots)} sample spots')
    
    # We apply the function that add records of missing days
    dataframe = add_missing_days(dataframe, 'PUNTO_MUESTREO')
    
    print(f'Dataframe shape: {dataframe.shape}')
    
    cols_dimensiones = ['PROVINCIA', 'MUNICIPIO', 'ESTACION', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA']
    
    print('Stacking dataframes')
    df_h = get_stacked_dataframe(
        dataframe, cols_remain=cols_dimensiones,
        cols_to_drop=[col for col in list(dataframe.columns) if col[0] == 'V']
    )
    
    df_v = get_stacked_dataframe(
        dataframe, cols_remain=cols_dimensiones,
        cols_to_drop=[col for col in list(dataframe.columns) if col[0] == 'H']
    )
    
    print('Joining dataframes')
    final_df = add_last_col_to_df(df_h, df_v)
    
    print('Renaming columns')
    final_df = final_df.rename(columns={'level_7': 'HORA', 0: 'NIVEL_NO2', 'new_col': 'VALIDADO'})
    
    print('Formatting HORA column')
    final_df['HORA'] = final_df['HORA'].apply(lambda x: int(x[-2:]))
    
    print('Assigning values to non validated records')
    final_df = assign_non_validated_values(final_df, 'PUNTO_MUESTREO')
    print(f'Final dataframe shape: {final_df.shape}')
    
    return final_df

In [10]:
def main(url, file_name, desired_data_path, file_type):
    
    download_from_url(url, file_name)
    
    extract_from_zip_folder(file_type, file_name, desired_data_path)
    
    data = pd.DataFrame()
    
    for file_name in os.listdir(desired_data_path):
        print(f'Working with {file_name}')
        
        try:
            monthly_data = pd.read_csv(f'{desired_data_path}/{file_name}', sep=';')
        except:
            monthly_data = pd.read_csv(f'{desired_data_path}\\{file_name}', sep=';')

        monthly_data = get_clean_df(monthly_data)
        data = data.append(monthly_data, ignore_index=True)
        
    print(data.shape)
    print(data.head())
    
    return data

In [11]:
url = 'https://datos.madrid.es/egob/catalogo/201200-10306314-calidad-aire-horario.zip'
file_name = 'pollution_data.zip'
desired_data_path = 'pollution_data'
file_type = 'csv'

finalDF = main(url, file_name, desired_data_path, file_type)

extracting abr_mo18.csv
extracting ago_mo18.csv
extracting dic_mo18.csv
extracting ene_mo18.csv
extracting feb_mo18.csv
extracting jul_mo18.csv
extracting jun_mo18.csv
extracting mar_mo18.csv
extracting may_mo18.csv
extracting nov_mo18.csv
extracting oct_mo18.csv
extracting sep_mo18.csv
Working with abr_mo18.csv
Dataframe shape: (4490, 56)
Keeping just NO2 data
Dataframe shape: (719, 55)
Adding missing days rows
24 sample spots
Checking sample spot: 28079059_8_8
Checking sample spot: 28079057_8_8
Checking sample spot: 28079008_8_8
Checking sample spot: 28079050_8_8
Checking sample spot: 28079038_8_8
Checking sample spot: 28079027_8_8
Checking sample spot: 28079054_8_8
Checking sample spot: 28079036_8_8
Checking sample spot: 28079039_8_8
Checking sample spot: 28079060_8_8
Checking sample spot: 28079055_8_8
Checking sample spot: 28079047_8_8
Checking sample spot: 28079024_8_8
Checking sample spot: 28079004_8_8
Checking sample spot: 28079056_8_8
Checking sample spot: 28079017_8_8
Checking

Checking sample spot: 28079017_8_8
Checking sample spot: 28079040_8_8
Checking sample spot: 28079048_8_8
Checking sample spot: 28079018_8_8
Checking sample spot: 28079016_8_8
Checking sample spot: 28079049_8_8
Checking sample spot: 28079058_8_8
Checking sample spot: 28079011_8_8
Checking sample spot: 28079035_8_8
Day 3-8-2018 missing
[28, 79, 35, '28079035_8_8', 2018, 8, 3, 97.0, 'N', 94.0, 'N', 80.0, 'N', 47.0, 'N', 45.0, 'N', 39.0, 'N', 30.0, 'N', 57.0, 'N', 70.0, 'N', 70.0, 'N', 57.0, 'N', 62.0, 'N', 58.0, 'N', 61.0, 'N', 39.0, 'N', 24.0, 'N', 35.0, 'N', 37.0, 'N', 57.0, 'N', 38.0, 'N', 61.0, 'N', 143.0, 'N', 185.0, 'N', 176.0, 'N']
Day 3-8-2018 row added to original dataframe
Day 4-8-2018 missing
[28, 79, 35, '28079035_8_8', 2018, 8, 4, 97.0, 'N', 94.0, 'N', 80.0, 'N', 47.0, 'N', 45.0, 'N', 39.0, 'N', 30.0, 'N', 57.0, 'N', 70.0, 'N', 70.0, 'N', 57.0, 'N', 62.0, 'N', 58.0, 'N', 61.0, 'N', 39.0, 'N', 24.0, 'N', 35.0, 'N', 37.0, 'N', 57.0, 'N', 38.0, 'N', 61.0, 'N', 143.0, 'N', 185.0,

Reassigning NO2 value for spot 28079035_8_8, 2018-8-6 24:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 1:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 2:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 3:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 4:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 5:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 6:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 7:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 8:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 9:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-7 10:00
Reassigning NO2 value for spot 28079035_8_8, 2018-8-30 10:00
Final dataframe shape: (17856, 10)
Working with dic_mo18.csv
Dataframe shape: (4679, 56)
Keeping just NO2 data
Dataframe shape: (744, 55)
Adding missing days rows
24 sample spots
Checking sample spot: 28079059_8_8
Checking sample spot: 28079057_8_8
Checking sample spot: 28079008_8_8
Chec

Reassigning NO2 value for spot 28079049_8_8, 2018-1-15 10:00
Reassigning NO2 value for spot 28079058_8_8, 2018-1-17 10:00
Reassigning NO2 value for spot 28079058_8_8, 2018-1-29 10:00
Reassigning NO2 value for spot 28079011_8_8, 2018-1-10 4:00
Reassigning NO2 value for spot 28079011_8_8, 2018-1-10 5:00
Reassigning NO2 value for spot 28079011_8_8, 2018-1-17 13:00
Reassigning NO2 value for spot 28079035_8_8, 2018-1-3 18:00
Reassigning NO2 value for spot 28079035_8_8, 2018-1-15 14:00
Reassigning NO2 value for spot 28079035_8_8, 2018-1-15 15:00
Reassigning NO2 value for spot 28079035_8_8, 2018-1-16 9:00
Reassigning NO2 value for spot 28079035_8_8, 2018-1-16 10:00
Reassigning NO2 value for spot 28079035_8_8, 2018-1-16 11:00
Reassigning NO2 value for spot 28079035_8_8, 2018-1-29 12:00
Final dataframe shape: (17856, 10)
Working with feb_mo18.csv
Dataframe shape: (4228, 56)
Keeping just NO2 data
Dataframe shape: (672, 55)
Adding missing days rows
24 sample spots
Checking sample spot: 28079059_8

Reassigning NO2 value for spot 28079035_8_8, 2018-7-22 20:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-22 21:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-22 22:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-22 23:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-22 24:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 1:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 2:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 3:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 4:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 5:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 6:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 7:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 8:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 9:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 10:00
Reassigning NO2 value for spot 28079035_8_8, 2018-7-23 11:00
Reassigning NO2 value for spot 28

Checking sample spot: 28079058_8_8
Checking sample spot: 28079011_8_8
Checking sample spot: 28079035_8_8
Dataframe shape: (744, 55)
Stacking dataframes
Joining dataframes
Renaming columns
Formatting HORA column
Assigning values to non validated records
Reassigning NO2 value for spot 28079059_8_8, 2018-3-1 9:00
Reassigning NO2 value for spot 28079059_8_8, 2018-3-25 3:00
Reassigning NO2 value for spot 28079057_8_8, 2018-3-25 3:00
Reassigning NO2 value for spot 28079057_8_8, 2018-3-27 10:00
Reassigning NO2 value for spot 28079008_8_8, 2018-3-12 14:00
Reassigning NO2 value for spot 28079008_8_8, 2018-3-25 3:00
Reassigning NO2 value for spot 28079050_8_8, 2018-3-9 4:00
Reassigning NO2 value for spot 28079050_8_8, 2018-3-9 5:00
Reassigning NO2 value for spot 28079050_8_8, 2018-3-9 6:00
Reassigning NO2 value for spot 28079050_8_8, 2018-3-9 7:00
Reassigning NO2 value for spot 28079050_8_8, 2018-3-9 8:00
Reassigning NO2 value for spot 28079050_8_8, 2018-3-9 9:00
Reassigning NO2 value for spot 2

Reassigning NO2 value for spot 28079040_8_8, 2018-5-30 16:00
Reassigning NO2 value for spot 28079048_8_8, 2018-5-30 18:00
Reassigning NO2 value for spot 28079048_8_8, 2018-5-30 19:00
Reassigning NO2 value for spot 28079018_8_8, 2018-5-8 11:00
Reassigning NO2 value for spot 28079018_8_8, 2018-5-21 11:00
Reassigning NO2 value for spot 28079018_8_8, 2018-5-21 12:00
Reassigning NO2 value for spot 28079018_8_8, 2018-5-23 10:00
Reassigning NO2 value for spot 28079016_8_8, 2018-5-10 10:00
Reassigning NO2 value for spot 28079016_8_8, 2018-5-23 13:00
Reassigning NO2 value for spot 28079016_8_8, 2018-5-23 14:00
Reassigning NO2 value for spot 28079049_8_8, 2018-5-28 12:00
Reassigning NO2 value for spot 28079049_8_8, 2018-5-28 13:00
Reassigning NO2 value for spot 28079058_8_8, 2018-5-7 18:00
Reassigning NO2 value for spot 28079058_8_8, 2018-5-16 11:00
Reassigning NO2 value for spot 28079058_8_8, 2018-5-16 12:00
Reassigning NO2 value for spot 28079058_8_8, 2018-5-21 18:00
Reassigning NO2 value for 

Reassigning NO2 value for spot 28079035_8_8, 2018-11-11 24:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 1:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 2:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 3:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 4:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 5:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 6:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 7:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 8:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 9:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 10:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 11:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 12:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 13:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-12 14:00
Reassigning NO2 value for spot 28079035_8_8, 2018-11-15 12:00
Reassigning NO2 v

Reassigning NO2 value for spot 28079036_8_8, 2018-9-6 4:00
Reassigning NO2 value for spot 28079036_8_8, 2018-9-12 13:00
Reassigning NO2 value for spot 28079036_8_8, 2018-9-27 13:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-3 22:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-3 23:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-3 24:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 1:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 2:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 3:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 4:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 5:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 6:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 7:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 8:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 9:00
Reassigning NO2 value for spot 28079039_8_8, 2018-9-4 10:00
Reassigning NO2 value for spot 28079039_8_8, 201

In [12]:
finalDF.head()

,PROVINCIA,MUNICIPIO,ESTACION,PUNTO_MUESTREO,ANO,MES,DIA,HORA,NIVEL_NO2,VALIDADO
0,28,79,4,28079004_8_8,2018,4,1,1,21.0,V
1,28,79,4,28079004_8_8,2018,4,1,2,19.0,V
2,28,79,4,28079004_8_8,2018,4,1,3,17.0,V
3,28,79,4,28079004_8_8,2018,4,1,4,24.0,V
4,28,79,4,28079004_8_8,2018,4,1,5,17.0,V


In [13]:
finalDF['MES'].value_counts()

12    17856
10    17856
8     17856
7     17856
5     17856
3     17856
1     17856
11    17280
9     17280
6     17280
4     17280
2     16128
Name: MES, dtype: int64

In [14]:
finalDF['ANO'].value_counts()

2018    210240
Name: ANO, dtype: int64

In [15]:
finalDF['VALIDADO'].value_counts()

V    209081
R      1159
Name: VALIDADO, dtype: int64